In [ ]:
!pip install transformers

In [ ]:
!wget https://raw.githubusercontent.com/huggingface/transformers/master/examples/question-answering/run_squad.py

!git clone https://github.com/dmis-lab/biobert.git

In [ ]:
# Default BioASQ test files doesn't match SQUAD format. So add dummy keys 
# to match it to make predictions.
with open("BioASQ/BioASQ/BioASQ-test-factoid-4b-1.json") as f:
    data = json.load(f)

new_paragraph = []
for questions in data["data"][0]["paragraphs"]:
    new_questions = []
    for question in questions["qas"]:
        new_question = {**question}
        # Adding dummy answer key.
        new_question["answers"] = [{"text": "", "answer_start": 0}]
        new_questions.append(new_question)
        
    new_paragraph.append({**questions, "qas": new_questions})
    
data["data"][0]["paragraphs"] = new_paragraph

with open("BioASQ/BioASQ/Answered_BioASQ-test-factoid-4b-1.json", "w") as f:
    json.dump(data, f)

In [ ]:
# train_files directory format
# train_files
#     train-v1.1.json
#     dev-v1.1.json
# For the predictions on the raw data
#     Answered_BioASQ-test-factoid-4b-1.json

In [ ]:
# google/electra-small-discriminator
# enelpi/med-electra-small-discriminator

In [ ]:
!python run_squad.py \
--model_type electra \
--model_name_or_path enelpi/med-electra-small-discriminator \
--do_train \
--do_eval \
--data_dir train_files \
--predict_file Answered_BioASQ-test-factoid-4b-1.json \
--cache_dir med_electra_cached_data \
--per_gpu_train_batch_size 16 \
--num_train_epochs 5 \
--max_seq_length 128 \
--doc_stride 128 \
--output_dir ./med_electra_finetuned_model

In [ ]:
!python biobert/biocodes/transform_nbset2bioasqform.py --nbest_path=med_electra_finetuned_model/nbest_predictions_.json --output_path=prediction_outputs

In [ ]:
!java -Xmx10G -cp $CLASSPATH:./flat/BioASQEvaluation/dist/BioASQEvaluation.jar evaluation.EvaluatorTask1b -phaseB -e 5 BioASQ/BioASQ/4B1_golden.json prediction_outputs